# Motion Prediction

### Libraries

In [1]:
#Importing libraries
import os
import sys
import xlsxwriter
from sklearn import preprocessing
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#Importing Data preprocessing functions
sys.path.append('data_processing/')
from readDataset import dataGrabber
from preProcessing import preProcess
from dataPreparation import dataPrepare

In [2]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"The device is: {device}")

The device is: cuda


### Load Dataset

Either based on location id or recording id

In [3]:
#Load the dataset based on recording location
#Then uncomment and run this cell. Skip next cell

dataset_path = 'dataset/data/'

"""
# Edit to some other number to load a different dataset
locations_sel = ['4']

# Initialize data Grabber Object
data_obj = dataGrabber(dataset_path)

data_obj.location_id = locations_sel
data_obj.read_csv_with_location()

track_data_raw = data_obj.get_tracks_data()
track_meta_data_raw = data_obj.get_tracksMeta_data()
"""

"\n# Edit to some other number to load a different dataset\nlocations_sel = ['4']\n\n# Initialize data Grabber Object\ndata_obj = dataGrabber(dataset_path)\n\ndata_obj.location_id = locations_sel\ndata_obj.read_csv_with_location()\n\ntrack_data_raw = data_obj.get_tracks_data()\ntrack_meta_data_raw = data_obj.get_tracksMeta_data()\n"

In [4]:
#Load the dataset based on recording id
#Then uncomment and run this cell. Skip previous cell

dataset_path = 'dataset/data/'

recording_id_sel = ['27']

# Initialize data Grabber Object
data_obj = dataGrabber(dataset_path)

data_obj.recording_id = recording_id_sel
data_obj.read_csv_with_recordingID()

track_data_raw = data_obj.get_tracks_data()
track_meta_data_raw = data_obj.get_tracksMeta_data()  


### Preprocess the data

In [5]:
'''
Data preprocessing using the class preProcess()
'''
pre_process_obj = preProcess()
pre_process_obj.tracks_data = track_data_raw
pre_process_obj.tracks_meta_data = track_meta_data_raw
pre_process_obj.recording_ids = data_obj.recording_id
pre_process_obj.data_len = len(track_data_raw)

'''
Downsampling Data

# Define the number of frames to be skipped + 1 => here 4 frames are skipped so 4+1 = 5
'''
pre_process_obj.frames_skipped = 5
track_data_downsampled, tracks_meta_data = pre_process_obj.get_down_sampled_data()
pre_process_obj.tracks_data = track_data_downsampled

'''
Encode objects (0: bicycle, 1 : car, 2 : pedestrian, 3 : truck_bus)
'''
pre_process_obj.label_encoding()
pre_process_obj.print_label_encoder_classes()

#Normalize data
tracks_data_norm, min_max_scalar_list = pre_process_obj.normalize_data()

Labels Encoded
0 : bicycle
1 : car
2 : pedestrian
3 : truck_bus


### Load data 

In [6]:
# Resetting dropped frames index
tracks_data_norm = tracks_data_norm.reset_index(drop=True)

'''
Data preperation using the class dataPrepare()
'''
data_prepare_obj = dataPrepare()
data_prepare_obj.tracks_data_norm = tracks_data_norm
data_prepare_obj.tracksMeta_data = tracks_meta_data
data_prepare_obj.data_len = len(tracks_data_norm)

'''
Choose the number of track id to be used and split the data
'''
# Number for track id to be used
data_prepare_obj.track_id_range = 100

data_prepare_obj.data_input = "normalized_data"
xTrain_data, xTest_data, yTrain_data, yTest_data = data_prepare_obj.get_test_train_split()

This might take a while!
Current progress: 0.0 %
Current progress: 1.01 %
Current progress: 2.02 %
Current progress: 3.03 %
Current progress: 4.04 %
Current progress: 5.05 %
Current progress: 6.06 %
Current progress: 7.07 %
Current progress: 8.08 %
Current progress: 9.09 %
Current progress: 10.1 %
Current progress: 11.11 %
Current progress: 12.12 %
Current progress: 13.13 %
Current progress: 14.14 %
Current progress: 15.15 %
Current progress: 16.16 %
Current progress: 17.17 %
Current progress: 18.18 %
Current progress: 19.19 %
Current progress: 20.2 %
Current progress: 21.21 %
Current progress: 22.22 %
Current progress: 23.23 %
Current progress: 24.24 %
Current progress: 25.25 %
Current progress: 26.26 %
Current progress: 27.27 %
Current progress: 28.28 %
Current progress: 29.29 %
Current progress: 30.3 %
Current progress: 31.31 %
Current progress: 32.32 %
Current progress: 33.33 %
Current progress: 34.34 %
Current progress: 35.35 %
Current progress: 36.36 %
Current progress: 37.37 %
C

### Load model, data and training

In [7]:
sys.path.append('models/')
from models import FcnTorch

In [8]:
#Reshape data

n_input = np.shape(xTrain_data)[1] * np.shape(xTrain_data)[2]
xTrain = np.reshape(xTrain_data, (np.shape(xTrain_data)[0], n_input))
n_output = np.shape(yTrain_data)[1] * np.shape(yTrain_data)[2]
yTrain = np.reshape(yTrain_data, (np.shape(yTrain_data)[0], n_output))

xTest = np.reshape(xTest_data, (np.shape(xTest_data)[0], n_input))
yTest = np.reshape(yTest_data, (np.shape(yTest_data)[0], n_output))

In [9]:
np.shape(yTrain)

(113467, 45)

In [10]:
#Convert to tensors and prepare dataloader
X_train = torch.FloatTensor(xTrain)
X_test = torch.FloatTensor(xTest)
y_train = torch.FloatTensor(yTrain)
y_test = torch.FloatTensor(yTest)

trainloader = DataLoader(X_train, batch_size=64, shuffle=True)
testloader = DataLoader(X_test, batch_size=64, shuffle=False)

In [11]:
#Check model
torch.manual_seed(33)
model = FcnTorch().to(device)
model

FcnTorch(
  (fc1): Linear(in_features=80, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=45, bias=True)
)

In [12]:
model.count_parameters()

  40960
    512
 131072
    256
  32768
    128
   8192
     64
   2048
     32
   1440
     45
_______
 217517 <- Total trainable parameters


In [13]:
#define loss function and optimizer

criterion = nn.MSELoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
#Training the model

import time
start_time = time.time()

epochs = 5000
losses = []

for i in range(epochs):
#     i+=1
    X_train, y_train = X_train.to(device), y_train.to(device)
    y_pred = model(X_train)
    
    loss = torch.sqrt(criterion(y_pred, y_train))
    losses.append(loss)
    
    #print losses for every 100 epochs
    if i%100 == 1:
        print(f'epoch: {i:3}  loss: {loss.item():10.6f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'epoch: {i:3}  loss: {loss.item():10.6f}') 
print(f'\nDuration: {time.time() - start_time:.0f} seconds') 

epoch:   1  loss:   0.580160
epoch: 101  loss:   0.076768
epoch: 201  loss:   0.027865
epoch: 301  loss:   0.019568
epoch: 401  loss:   0.017327
epoch: 501  loss:   0.017452
epoch: 601  loss:   0.016111
epoch: 701  loss:   0.016537
epoch: 801  loss:   0.016106
epoch: 901  loss:   0.015660
epoch: 1001  loss:   0.015019
epoch: 1101  loss:   0.014866
epoch: 1201  loss:   0.015211
epoch: 1301  loss:   0.013901
epoch: 1401  loss:   0.013854
epoch: 1501  loss:   0.013170
epoch: 1601  loss:   0.013085
epoch: 1701  loss:   0.012336
epoch: 1801  loss:   0.012290
epoch: 1901  loss:   0.012219
epoch: 2001  loss:   0.011819
epoch: 2101  loss:   0.012010
epoch: 2201  loss:   0.012022
epoch: 2301  loss:   0.011039
epoch: 2401  loss:   0.012372
epoch: 2501  loss:   0.011334
epoch: 2601  loss:   0.010912
epoch: 2701  loss:   0.011374
epoch: 2801  loss:   0.011602
epoch: 2901  loss:   0.010664
epoch: 3001  loss:   0.010412
epoch: 3101  loss:   0.010921
epoch: 3201  loss:   0.011053
epoch: 3301  loss:  

In [15]:
X_train, y_train = X_train.cpu(), y_train.cpu()
y_pred = y_pred.cpu().detach().numpy()
model = model.cpu()

### Generating Test dataset

In [16]:
work_book_filename = 'NN_velocity_prediction_result.xlsx' 

In [17]:
# Based on recording id

recording_id_sel = ['28']
data_sel_id = 0 # If there are multiple recordings added

# Initialize data Grabber Object
test_data_obj = dataGrabber(dataset_path)

test_data_obj.recording_id = recording_id_sel
test_data_obj.read_csv_with_recordingID()

test_track_data = test_data_obj.get_tracks_data()
test_track_meta_data = test_data_obj.get_tracksMeta_data() 

Preprocessing and downsampling data to match training dataset

In [18]:
test_pre_process_obj = preProcess()
test_pre_process_obj.tracks_data = test_track_data
test_pre_process_obj.tracks_meta_data = test_track_meta_data
test_pre_process_obj.recording_ids = test_data_obj.recording_id
test_pre_process_obj.data_len = len(test_track_data)

test_pre_process_obj.frames_skipped = 5
test_track_data_downsampled, test_tracks_meta_data = test_pre_process_obj.get_down_sampled_data()
test_pre_process_obj.tracks_data = test_track_data_downsampled

Data normalization

In [19]:
# Gets the tracks data normalized
test_tracks_data_norm, min_max_scalar_list = test_pre_process_obj.normalize_data()
# Resetting dropped frames index
test_tracks_data_norm = test_tracks_data_norm.reset_index(drop=True)

# Saving Normalized Data
test_data_prepare_obj = dataPrepare()
test_data_prepare_obj.tracks_data_norm = test_tracks_data_norm
test_data_prepare_obj.tracksMeta_data = test_tracks_meta_data
test_data_prepare_obj.data_len = len(test_tracks_data_norm)

Data stacking

In [20]:
# Number for track id to be used
test_data_prepare_obj.track_id_range = 10  

# Gets the tracks data normalized and its ID
test_data_prepare_obj.data_input = "normalized_data"
t_norm_Ids, t_in_norm, t_out_norm = test_data_prepare_obj.data_stacking()
# Predict the output
n_input = np.shape(t_in_norm)[1] * np.shape(t_in_norm)[2]
t_in_norm_reshaped = np.reshape(t_in_norm, (np.shape(t_in_norm)[0], n_input))

This might take a while!
Current progress: 0.0 %
Current progress: 11.11 %
Current progress: 22.22 %
Current progress: 33.33 %
Current progress: 44.44 %
Current progress: 55.56 %
Current progress: 66.67 %
Current progress: 77.78 %
Current progress: 88.89 %
Current progress: 100.0 %
Done! 


### Evaluation

In [21]:
sys.path.append('evaluation/')
from pred_evaluator import Evaluation
from evaluation_matrix import evaluationMatrix

In [22]:
data_eval_obj = Evaluation()

# Resetting dropped frames index
test_track_data_downsampled = test_track_data_downsampled.reset_index(drop=True)
ground_truth_prepare_obj = dataPrepare()
ground_truth_prepare_obj.data_input = "raw_data"
ground_truth_prepare_obj.track_id_range = 10
ground_truth_prepare_obj.tracksMeta_data = test_tracks_meta_data
ground_truth_prepare_obj.tracks_data_norm = test_tracks_data_norm
ground_truth_prepare_obj.tracks_data = test_track_data_downsampled
ground_truth_prepare_obj.data_len = len(test_track_data_downsampled) 
#ground_truth_prepare_obj.num_predict = 15
t_raw_Ids, t_in_raw, t_out_raw = ground_truth_prepare_obj.data_stacking()

This might take a while!
Current progress: 0.0 %
Current progress: 11.11 %
Current progress: 22.22 %
Current progress: 33.33 %
Current progress: 44.44 %
Current progress: 55.56 %
Current progress: 66.67 %
Current progress: 77.78 %
Current progress: 88.89 %
Current progress: 100.0 %
Done! 


In [23]:
data_eval_obj.t_raw_Ids = t_raw_Ids
data_eval_obj.t_in_raw = t_in_raw
data_eval_obj.t_out_raw = t_out_raw

In [24]:
xCenter_gt, yCenter_gt, heading_gt = data_eval_obj.get_ground_truth()

In [25]:
yhat = model(torch.FloatTensor(t_in_norm_reshaped))
    
# Save Predicted Data into the Evaluator
data_eval_obj.y_hat = yhat.detach().numpy()

In [26]:
# Set Paramters
data_eval_obj.min_max_scalar_list = min_max_scalar_list

# Get Prediction
xCenter_prediction, yCenter_prediction, heading_prediction = data_eval_obj.get_prediction()

Storing the generated predictions in a Excel file

In [27]:
#Delete the file if it already exists
if os.path.exists(work_book_filename):
    os.remove(work_book_filename)

In [28]:
data_eval_obj.wb_filename = work_book_filename
data_eval_obj.write_to_workbook()

ade -- Average displacement error

fde -- Final displacement error

ahe -- Average absolute heading error

In [29]:
eval_obj = evaluationMatrix(work_book_filename, data_eval_obj.n_predict)
ade_value, fde_value, ahe_val = eval_obj.get_fde_ade_ahe()

The average displacement error is 2.267 m
The average final displacement error is 2.021 m
The average absolute heading error is 4.72 degrees


Save the trained model

In [30]:
# torch.save(model.state_dict(), 'PathPredModel.pt')